In [2]:
pip install soda-core==3.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.8/196.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.8/431.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.7/221.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.18.5
    Uninstalling ruamel.yaml-0.18.5:
      Successfully uninstalled ruamel.yaml-0.18.5
  Attempting uninstall: opente

In [3]:
!pip install soda-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.8/470.8 kB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 91.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... - \ | done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 103.1 MB/s eta 0:00:0000:0100:01
  P

In [4]:
!pip install soda-core-duckdb

In [5]:
!soda --version

soda-core, version 3.4.3


Connect DuckDB and Soda to run data quality checks and create a txt report

In [7]:
import soda
from soda import scan
import pandas as pd
import duckdb
import os

# Define the path to the files in OneLake
csv_file_path = '/lakehouse/default/Files/sample_data_quality_test_large.csv'
soda_yaml_path = '/lakehouse/default/Files/soda_duckdb/checks.yml'
config_yaml_path = '/lakehouse/default/Files/soda_duckdb/config.yml'  # Path to your config.yml file

# Set the environment variable for the Soda configuration file
os.environ['SODA_CONFIG_FILE'] = config_yaml_path

# Load the CSV file into a Pandas DataFrame
data_df = pd.read_csv(csv_file_path)
#print(f"sample data: {data_df}")

# Connect to DuckDB and create the table
conn = duckdb.connect("my_database.myduckdb")
conn.sql('''CREATE TABLE IF NOT EXISTS sample_data AS SELECT * FROM data_df''')

# Show the count of rows in the table (optional)
row_count = conn.sql('SELECT COUNT(*) FROM sample_data').fetchone()[0]
print(f"Row count in sample_data table: {row_count}")

# Initialize and configure the Soda scan
scan = Scan()
scan.add_duckdb_connection(conn)
scan.set_data_source_name("duckdb")  # Match the data source name in your config.yml
scan.add_sodacl_yaml_files(soda_yaml_path)
scan.set_scan_definition_name("test_destinations")
scan.execute()
#scan.assert_no_checks_fail()

# Generate and save the report
report_text = scan.get_logs_text()
with open('/lakehouse/default/Files/soda_duckdb/report.txt', 'w') as report_file:
    report_file.write(report_text)

print("Report generated and saved to /lakehouse/default/Files/soda_duckdb/report.txt")

print(scan.get_logs_text())

# Close the connection
conn.close()

Row count in sample_data table: 1000
Report generated and saved to /lakehouse/default/Files/soda_duckdb/report.txt
INFO   | Soda Core 3.4.3
INFO   | Using DefaultSampler
INFO   | Scan summary:
INFO   | 3/4 checks PASSED: 
INFO   |     sample_data in duckdb
INFO   |       row_count > 0 [PASSED]
INFO   |       missing_count(CustomerID) = 0 [PASSED]
INFO   |       duplicate_count(CustomerID) = 0 [PASSED]
INFO   | 1/4 checks FAILED: 
INFO   |     sample_data in duckdb
INFO   |       invalid_count(Email) = 0 [FAILED]
INFO   |         check_value: 66
INFO   | Oops! 1 failures. 0 warnings. 0 errors. 3 pass.


Connect DuckDB and Soda to run data quality checks and save the result in duck db

In [31]:
from soda.scan import Scan
import pandas as pd
import duckdb
import os

# Define the path to the files in OneLake
csv_file_path = '/lakehouse/default/Files/sample_data_quality_test_large.csv'
soda_yaml_path = '/lakehouse/default/Files/soda_duckdb/checks.yml'
config_yaml_path = '/lakehouse/default/Files/soda_duckdb/config.yml'  # Path to your config.yml file

# Set the environment variable for the Soda configuration file
os.environ['SODA_CONFIG_FILE'] = config_yaml_path

# Load the CSV file into a Pandas DataFrame
data_df = pd.read_csv(csv_file_path)

# Connect to DuckDB and create the table
conn = duckdb.connect("my_database.myduckdb")
conn.sql('''CREATE TABLE IF NOT EXISTS sample_data AS SELECT * FROM data_df''')

# Show the count of rows in the table (optional)
row_count = conn.sql('SELECT COUNT(*) FROM sample_data').fetchone()[0]
print(f"Row count in sample_data table: {row_count}")

# Initialize and configure the Soda scan
scan = Scan()
scan.add_duckdb_connection(conn)
scan.set_data_source_name("duckdb")  # Match the data source name in your config.yml
scan.add_sodacl_yaml_files(soda_yaml_path)
scan.set_scan_definition_name("test_destinations")
scan.execute()

# Generate and save the report
report_text = scan.get_logs_text()

# Parse the logs and store them in DuckDB
log_lines = report_text.split('\n')
log_data = {'Timestamp': [], 'Level': [], 'Message': []}
for line in log_lines:
    if line:
        parts = line.split(' ', 2)
        log_data['Timestamp'].append(parts[0])
        log_data['Level'].append(parts[1])
        log_data['Message'].append(parts[2])

log_df = pd.DataFrame(log_data)

# Create the structured table in DuckDB
conn.sql('''
    CREATE TABLE IF NOT EXISTS data_quality_results (
        Timestamp TIMESTAMP,
        Level VARCHAR,
        Message TEXT
    )
''')

# Insert the data into the structured table
conn.register('log_df', log_df)
conn.sql('INSERT INTO data_quality_results SELECT * FROM log_df')

print("Data quality results stored in DuckDB")

# Close the connection
#conn.close()

#WORKING

Row count in sample_data table: 1000
Data quality results stored in DuckDB


In [32]:
conn.sql('select * from data_quality_results')


┌───────────┬─────────┬────────────────────────────────────────────────────┐
│ Timestamp │  Level  │                      Message                       │
│  varchar  │ varchar │                      varchar                       │
├───────────┼─────────┼────────────────────────────────────────────────────┤
│ INFO      │         │  | Soda Core 3.4.3                                 │
│ INFO      │         │  | Using DefaultSampler                            │
│ INFO      │         │  | Scan summary:                                   │
│ INFO      │         │  | 3/4 checks PASSED:                              │
│ INFO      │         │  |     sample_data in duckdb                       │
│ INFO      │         │  |       row_count > 0 [PASSED]                    │
│ INFO      │         │  |       missing_count(CustomerID) = 0 [PASSED]    │
│ INFO      │         │  |       duplicate_count(CustomerID) = 0 [PASSED]  │
│ INFO      │         │  | 1/4 checks FAILED:                              │

Use the DuckDB to create the csv result

In [16]:
# Reconnect to DuckDB
conn = duckdb.connect("my_database.myduckdb")

# Export the data quality results to a CSV file
conn.sql('COPY data_quality_results TO \'/lakehouse/default/Files/soda_duckdb/data_quality_results.csv\' (HEADER, DELIMITER \',\')')

print("Data quality results exported to CSV")

# Close the connection
conn.close()

Data quality results exported to CSV


For reporting via Notebook use below.. NOT TESTED

In [8]:
!pip install dash plotly pandas

In [10]:
import os
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Create a DataFrame from the scan logs for visualization
log_lines = report_text.split('\n')
log_data = {'Timestamp': [], 'Level': [], 'Message': []}
for line in log_lines:
    if line:
        parts = line.split(' ', 2)
        log_data['Timestamp'].append(parts[0])
        log_data['Level'].append(parts[1])
        log_data['Message'].append(parts[2])

log_df = pd.DataFrame(log_data)

# Create a Dash app
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Data Quality Dashboard'),

    dcc.Graph(
        id='log-levels',
        figure=px.histogram(log_df, x='Level', title='Log Levels Distribution')
    ),

    dcc.Graph(
        id='log-messages',
        figure=px.scatter(log_df, x='Timestamp', y='Level', text='Message', title='Log Messages Over Time')
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)